In [61]:
import os
import glob
from datetime import datetime
#import warnings
#warnings.simplefilter('ignore')
import scipy as sp
import scipy.ndimage
import numpy as np
import pandas as pd
import skimage
import skimage.exposure
import mahotas as mh
from sklearn.cross_validation import KFold
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from tqdm import tqdm_notebook
from IPython.display import display
# from dual_IDG import DualImageDataGenerator

In [62]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, \
    Convolution2D, MaxPooling2D, ZeroPadding2D, Input, Embedding, LSTM, merge, \
    Lambda, UpSampling2D, Deconvolution2D, Cropping2D
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [63]:
from keras.losses import binary_crossentropy
def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop

from losses import *

def get_unet_128(input_shape=(128, 128, 3),
                 num_classes=1):
    inputs = Input(shape=input_shape)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(inputs)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    # 128

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up1)

    model = Model(inputs=inputs, outputs=classify)

    model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
    return model

In [64]:
model = get_unet_128()
# model.compile(optimizer=SGD(lr=1e-3, momentum=0.95),
#               loss=log_dice_loss,
#               metrics=[mean_IOU_gpu, dice_metric])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 128, 128, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_111 (BatchN (None, 128, 128, 64) 256         conv2d_116[0][0]                 
__________________________________________________________________________________________________
activation_111 (Activation)     (None, 128, 128, 64) 0           batch_normalization_111[0][0]    
__________________________________________________________________________________________________
conv2d_117

In [65]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [66]:
filelist = glob.glob('dataset/crop/RIM-ONEv2/images/*.jpg')              #change '/'
X = np.array([np.array(Image.open(fname)) for fname in filelist])
filelist = glob.glob('dataset/crop/RIM-ONEv1/images/*.jpg')              #change '/'
X = np.concatenate((X,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/DRIONS_DB/images/*.jpg')              #change '/'
X = np.concatenate((X,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/DRISHTI_GS/images/*.jpg')              #change '/'
X = np.concatenate((X,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/RIM-ONEv3/images/*.jpg')              #change '/'
X = np.concatenate((X,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
# X=X.reshape(841,3,128,128)
print(X.shape)
# X = np.concatenate((glu,nonglu),axis=0)
# print(X.shape)
# print(X[1,:].shape)


(841, 128, 128, 3)


In [67]:
filelist = glob.glob('dataset/crop/RIM-ONEv2/discs/*.png')              #change '/'
Y = np.array([np.array(Image.open(fname)) for fname in filelist])
filelist = glob.glob('dataset/crop/RIM-ONEv1/discs/*.png')              #change '/'
Y = np.concatenate((Y,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/DRIONS_DB/discs/*.png')              #change '/'
Y = np.concatenate((Y,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/DRISHTI_GS/discs/*.png')              #change '/'
Y = np.concatenate((Y,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
filelist = glob.glob('dataset/crop/RIM-ONEv3/discs/*.png')              #change '/'
Y = np.concatenate((Y,[np.array(Image.open(fname)) for fname in filelist]),axis=0)
Y = Y.reshape(841,128,128,1)
print(Y.shape)

(841, 128, 128, 1)


In [68]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.20, random_state=42)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,X_val.shape, y_val.shape)


(537, 128, 128, 3) (169, 128, 128, 3) (537, 128, 128, 1) (169, 128, 128, 1) (135, 128, 128, 3) (135, 128, 128, 1)


In [69]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
TensorBoard(log_dir='logs')]

/home/subodh/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [60]:

epochs = 50
batch_size = 32
H = model.fit(X_train, y_train, steps_per_epoch=np.ceil(float(len(X_train)) / float(batch_size)),
              epochs=epochs, verbose=2,
              callbacks=callbacks,validation_data=(X_val, y_val))

NameError: name 'model' is not defined

In [57]:
# model.fit(X_train, y_train, epochs=1, batch_size=1,shuffle=True,callbacks=callbacks,validation_data=(X_val, y_val))
scores = model.evaluate(X_test, y_test, batch_size=1, steps=None)
print(scores)

169/169 [==============================] - 575s 3s/step
[-986.60794532687, 1.6759058572131502]


In [70]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('first_model.h5')

# Deletes the existing model
del model  

# Returns a compiled model identical to the previous one
model = load_model('first_model.h5')

ValueError: Unknown loss function:bce_dice_loss

In [ ]:
import matplotlib.pyplot as plt
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Training for ' +str(50)+ ' epochs')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()

In [ ]:
plt.plot(H.history['acc'])
plt.plot(H.history['val_acc'])
plt.title('Training for ' +str(50)+ ' epochs')
plt.legend(['Training accuracy', 'Validation accuracy'], loc='lower right')
plt.show()